# Reconhecimentos de Padrões

## Alunos: Henrique Ricardo Figueira, Higor Celante
## Dataset: sEMG for Basic Hand movements Data Set


A base de dados utilizada tem as seguintes características:

- Série Temporal
- 2 canais EMG
- 5 arquivos

Datasets:
 - 6s = 2 homens e 2 mulheres na fixa dos 22 anos executaram os 6 movimentos 30 vezes cada, cada movimento sendo coletado por 6s.
 - 5s = 1 homem executou os 6 movimentos 100 vezes cada, por 5s cada movimento.
 
Movimentos:

- Esférico
- Palma aberta
- Lateral
- Cilíndrico
- Gancho
- Pinça


![movimentos](https://i.imgur.com/0faxxKu.png)

### Importações

In [1]:
import scipy.io
import numpy as np
import os
from librosa import stft
from numpy import mean, sqrt, square, arange
from sklearn import svm

from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OutputCodeClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.signal import welch
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,f1_score)

#### Carregando o Dataset

In [2]:
caminhos = [os.path.join("src", nome) for nome in os.listdir("src")]
pessoas = []

for essive in caminhos:
    if essive != "/novo.txt":
        mat = scipy.io.loadmat(essive)
        tip = [mat['tip_ch1'], mat['tip_ch2']]
        spher = [mat['spher_ch1'], mat['spher_ch2']]
        palmar = [mat['palm_ch1'], mat['palm_ch2']]
        lateral = [mat['lat_ch1'], mat['lat_ch2']]
        cilindrical = [mat['cyl_ch1'], mat['cyl_ch2']]
        hook = [mat['hook_ch1'], mat['hook_ch2']]
        data = [tip, spher, palmar, lateral, cilindrical, hook]
        data = np.array(data)
        pessoas.append(data)  
        




#print(" 6 movimentos,30 tentativas, 2 canais,  3000 coletas")


### Selecionando 1 dos Datasets

In [3]:
dataone = pessoas[2]
dataone = np.array(dataone)
dataone = np.swapaxes(dataone,1,2)

#### Shape após carregamento

> 6 movimentos, 30 tentativas, 2 canais,  3000 coletas

In [4]:
print(dataone.shape)

(6, 30, 2, 3000)


#### Segmentação

In [5]:
datax = []
segmentosize = (len(dataone[0,0,0,:]))/6
salto = segmentosize * 0.7
antpasso = segmentosize - salto
print(dataone.shape)
#dataone = dataone.reshape(6,30,2,25,)

print("Sobreposicao:        ", antpasso)
print("Salto:               ",salto)
print("Tamanho do segmento: ",segmentosize)


for movimento in dataone:  
    tentativs = []
    for tentativas in movimento:
        canals = []
        for canal in tentativas:
            
            listinha = []
            listinha.append(canal[350:850])
            listinha.append(canal[700:1200])
            listinha.append(canal[1050:1550])
            listinha.append(canal[1400:1900])
            listinha.append(canal[1750:2250])
            listinha.append(canal[2100:2600])
            canals.append(np.split(canal,6) + listinha)
        tentativs.append(canals)
    datax.append( tentativs)
datax = np.array(datax)


(6, 30, 2, 3000)
Sobreposicao:         150.0
Salto:                350.0
Tamanho do segmento:  500.0


![segmentacao](https://i.imgur.com/rWhhCop.png)


#### Shape após segmentação

##### 6 movimentos, 30 tentativas, 2 canais,  11 segmentos de 500 valores

In [6]:
print(datax.shape)
datax = np.swapaxes(datax, 0,1)
datax = datax.reshape(30,6,2,25,240)
print(datax.shape)

(6, 30, 2, 12, 500)
(30, 6, 2, 25, 240)


In [7]:
def vaiSVC(X,y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.4, shuffle=True)


    for kernel in ['rbf']:#, 'linear']:
        for gamma in [0.001, 0.01, 0.1]:
            for C in [1, 10, 100, 1000]:
                classificador = []
                classificador = svm.SVC(gamma=gamma, C=C, kernel=kernel).fit(X_train, y_train)
                print('acuracia:', (classificador.score(X_test, y_test)) * 100, 'kernel:', kernel, 'gamma:', gamma, 'C:', C)
            

    cls = []
    cls = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', n_components=7).fit(X_train, y_train)
    print('\n acuracia:', cls.score(X_test, y_test) * 100)

# Parte 1 -  Usando features como característica

## Parte 1.1 - Domínio do tempo

In [8]:
iemg = []
rmslist = []
varlist = []


for tentativa in datax:
    tmg = []
    tms = []
    tar = []
    for movimento in tentativa:
        mmg = []
        mms = []
        mar = []
        for canal in movimento:
            cmg = []
            cms = []
            car = []
            for segmento in canal:
                cmg.append((abs(segmento.sum())/ 500 ))
                cms.append(sqrt(mean(square(segmento))))
                car.append(np.var(segmento))
                
                
            mmg.append(cmg)
            mms.append(cms)
            mar.append(car)
        tmg.append(mmg)
        tms.append(mms)
        tar.append(mar)
    iemg.append(tmg)
    rmslist.append(tms)
    varlist.append(tar)
iemg = np.array(iemg)
rmslist = np.array(rmslist)
varlist = np.array(varlist)


# Trecho dedicado a pt 2
#print(auxiemg.shape)
#auxiemg = iemg.swapaxes(1,3)
#print(auxiemg.shape)
#auxrmslist = rmslist.swapaxes(1,3)
#auxvarlist = varlist.swapaxes(1,3)
#print(auxiemg.shape)
#auxiemg = auxiemg.reshape(30*2*12,6)
#auxrmslist = auxrmslist.reshape(30*2*12,6)
#auxvarlist = auxvarlist.reshape(30*2*12,6)

#pt2list = np.concatenate((auxrmslist,auxvarlist,auxiemg),axis = 3)
#print("shape para pt2", pt2list.shape)


######

iemg = np.swapaxes(iemg,2,3)

rmslist= np.swapaxes(rmslist,2,3)
print(rmslist.shape)
varlist = np.swapaxes(varlist,2,3)
iemg = iemg.reshape(30*6*25,2)
rmslist = rmslist.reshape(30*6*25,2)
varlist = varlist.reshape(30*6*25,2)
print(iemg.shape)

(30, 6, 25, 2)
(4500, 2)


### MAV

In [9]:

print(iemg.shape)

(4500, 2)


### RMS

In [10]:

print(rmslist.shape)

(4500, 2)


### VAR

In [11]:

print(varlist.shape)

(4500, 2)


In [12]:
#MRV = np.concatenate((rmslist,varlist,iemg),axis = 0)
MRV = [rmslist] + [varlist] + [iemg]
MRV = np.array(MRV)
print(MRV.shape)
MRV = MRV.swapaxes(0,1)

MRV = MRV.reshape(4500,3*2)
print(MRV.shape)

(3, 4500, 2)
(4500, 6)


### Criando o vetor de labels

In [13]:
X = MRV
y = np.array([[str(i)] * int(X.shape[0] / 6) for i in range(6)])
y = y.reshape(y.shape[0] * y.shape[1])
#y = y.swapaxes(0,1)
y.shape


(4500,)

### Treinando no domínio do tempo

In [14]:
vaiSVC(MRV,y)

acuracia: 16.27777777777778 kernel: rbf gamma: 0.001 C: 1
acuracia: 17.833333333333336 kernel: rbf gamma: 0.001 C: 10
acuracia: 18.61111111111111 kernel: rbf gamma: 0.001 C: 100
acuracia: 18.555555555555557 kernel: rbf gamma: 0.001 C: 1000
acuracia: 17.72222222222222 kernel: rbf gamma: 0.01 C: 1
acuracia: 18.555555555555557 kernel: rbf gamma: 0.01 C: 10
acuracia: 18.666666666666668 kernel: rbf gamma: 0.01 C: 100
acuracia: 18.166666666666668 kernel: rbf gamma: 0.01 C: 1000
acuracia: 18.61111111111111 kernel: rbf gamma: 0.1 C: 1
acuracia: 19.88888888888889 kernel: rbf gamma: 0.1 C: 10
acuracia: 19.611111111111114 kernel: rbf gamma: 0.1 C: 100
acuracia: 22.333333333333332 kernel: rbf gamma: 0.1 C: 1000

 acuracia: 18.11111111111111


/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(6, 6 - 1) = 5 components.
  ChangedBehaviorWarning)
/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


## Parte 1.2 -  Domínio da frequência

### Transformação domínio da frequência  /> FFT

In [15]:

data2 = np.swapaxes(datax, 1, 0)
data2 = data2.reshape(6,30,2,12*500)
print(data2.shape)
from scipy.signal import stft
_, _, w = stft(data2, fs=500, nperseg=512, noverlap=256)
w = np.swapaxes(w, 3, 4)

print(w.shape)

(6, 30, 2, 6000)
(6, 30, 2, 25, 257)


### FMD

In [16]:
def PSD(x):
    return np.sqrt(np.abs(x))

fmd = np.sum(PSD(w), axis=-1) / 2
#fmd = fmd.reshape(6*30*2*41)
print(fmd.shape)

(6, 30, 2, 25)


### MMDF

In [17]:
mmdf = np.sum(np.abs(w), axis=-1) / 2
#mmdf = mmdf.reshape(6*30*41*2)
print(mmdf.shape)

(6, 30, 2, 25)


#### Unindo as características no domínio da frequência

In [20]:


#FM = np.concatenate((fmd,mmdf), axis =-1)
FM = [fmd] + [mmdf]
FM = np.array(FM)
print(FM.shape)
FM = FM.swapaxes(0,-1)
FM = FM.swapaxes(0,2)
print(FM.shape)
FM = FM.reshape(6*30*25,2*2)
print(FM.shape)

(2, 6, 30, 2, 25)
(30, 6, 25, 2, 2)
(4500, 4)


### Criando o vetor de labels

In [21]:
X = FM
y = np.array([[str(i)] * int(X.shape[0] / 6) for i in range(6)])
y = y.reshape(y.shape[0] * y.shape[1])
y.shape

(4500,)

### Treinando e testando

In [22]:
vaiSVC(FM,y)

acuracia: 16.27777777777778 kernel: rbf gamma: 0.001 C: 1
acuracia: 18.11111111111111 kernel: rbf gamma: 0.001 C: 10
acuracia: 26.38888888888889 kernel: rbf gamma: 0.001 C: 100
acuracia: 28.61111111111111 kernel: rbf gamma: 0.001 C: 1000
acuracia: 26.833333333333332 kernel: rbf gamma: 0.01 C: 1
acuracia: 29.27777777777778 kernel: rbf gamma: 0.01 C: 10
acuracia: 32.22222222222222 kernel: rbf gamma: 0.01 C: 100
acuracia: 32.611111111111114 kernel: rbf gamma: 0.01 C: 1000
acuracia: 34.61111111111111 kernel: rbf gamma: 0.1 C: 1
acuracia: 36.666666666666664 kernel: rbf gamma: 0.1 C: 10
acuracia: 37.111111111111114 kernel: rbf gamma: 0.1 C: 100
acuracia: 38.55555555555556 kernel: rbf gamma: 0.1 C: 1000

 acuracia: 16.72222222222222


/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(4, 6 - 1) = 4 components.
  ChangedBehaviorWarning)
/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [23]:
#FM = FM[:12960,:]

print(FM.shape,MRV.shape)

(4500, 4) (4500, 6)


# Parte 1.3 -  Domínio da Frequência + Domínio do tempo

In [25]:
Combo = np.concatenate((FM,MRV), axis= 1)

#Combo = [FM] + [MRV]
Combo = np.array(Combo)
print(Combo.shape)

(4500, 10)


### Criando vetor de labels

In [26]:
X = Combo
y = np.array([[str(i)] * int(X.shape[0] / 6) for i in range(6)])
y = y.reshape(y.shape[0] * y.shape[1])
y.shape

(4500,)

### Treinando e testando

In [27]:
vaiSVC(Combo,y)

acuracia: 16.055555555555557 kernel: rbf gamma: 0.001 C: 1
acuracia: 18.944444444444443 kernel: rbf gamma: 0.001 C: 10
acuracia: 21.38888888888889 kernel: rbf gamma: 0.001 C: 100
acuracia: 25.0 kernel: rbf gamma: 0.001 C: 1000
acuracia: 23.77777777777778 kernel: rbf gamma: 0.01 C: 1
acuracia: 26.611111111111114 kernel: rbf gamma: 0.01 C: 10
acuracia: 28.444444444444443 kernel: rbf gamma: 0.01 C: 100
acuracia: 31.333333333333336 kernel: rbf gamma: 0.01 C: 1000
acuracia: 32.27777777777778 kernel: rbf gamma: 0.1 C: 1
acuracia: 34.22222222222222 kernel: rbf gamma: 0.1 C: 10
acuracia: 38.0 kernel: rbf gamma: 0.1 C: 100
acuracia: 40.11111111111111 kernel: rbf gamma: 0.1 C: 1000

 acuracia: 17.333333333333336


/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(10, 6 - 1) = 5 components.
  ChangedBehaviorWarning)
/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


# Tudo errado pra baixo

# Tudo errado pra baixo

# Tudo errado pra baixo

# Tudo errado pra baixo

# Experimento 1

## testar com segmentos maiores

In [194]:
print(datax.shape)

(30, 6, 2, 25, 240)


In [234]:
datax = datax.reshape(30, 6, 2,10,600)
#datax = datax.swapaxes(2,3)
print(datax.shape)

(30, 6, 2, 10, 600)


In [238]:
fmd = np.sum(PSD(datax), axis=-1) / 2
#fmd = fmd.reshape(30*6,2,10)
print(fmd.shape)

(30, 6, 2, 10)


In [239]:
mmdf = np.sum(np.abs(datax), axis=-1) / 2
#mmdf = mmdf.reshape(30*6,2*10)
print(mmdf.shape)

(30, 6, 2, 10)


In [243]:
UnionFreq = [mmdf] + [fmd]
UnionFreq = np.array(UnionFreq)
print(UnionFreq.shape)
UnionFreq = UnionFreq.swapaxes(0,2)
UnionFreq = UnionFreq.swapaxes(-1,2)
print(UnionFreq.shape)
UnionFreq = UnionFreq.reshape(6*30*10,2*2)
print(UnionFreq.shape)

(2, 30, 6, 2, 10)
(6, 30, 10, 2, 2)
(1800, 4)


In [244]:
X = UnionFreq
y = np.array([[str(i)] * int(X.shape[0] / 6) for i in range(6)])
y = y.reshape(y.shape[0] * y.shape[1])
y.shape

(1800,)

In [245]:
vaiSVC(X,y)

acuracia: 55.13888888888889 kernel: rbf gamma: 0.001 C: 1
acuracia: 55.97222222222222 kernel: rbf gamma: 0.001 C: 10
acuracia: 57.49999999999999 kernel: rbf gamma: 0.001 C: 100
acuracia: 56.80555555555556 kernel: rbf gamma: 0.001 C: 1000
acuracia: 53.75 kernel: rbf gamma: 0.01 C: 1
acuracia: 55.97222222222222 kernel: rbf gamma: 0.01 C: 10
acuracia: 55.69444444444444 kernel: rbf gamma: 0.01 C: 100
acuracia: 54.72222222222223 kernel: rbf gamma: 0.01 C: 1000
acuracia: 53.47222222222222 kernel: rbf gamma: 0.1 C: 1
acuracia: 51.24999999999999 kernel: rbf gamma: 0.1 C: 10
acuracia: 50.55555555555556 kernel: rbf gamma: 0.1 C: 100
acuracia: 50.83333333333333 kernel: rbf gamma: 0.1 C: 1000

 acuracia: 42.5


/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(4, 6 - 1) = 4 components.
  ChangedBehaviorWarning)
/home/higor/Documentos/RP/lib/python3.5/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


# Pra baixo nao corrigido

# Parte 2 - Usando movimentos como classe

## Parte 2.1 - Domnínio do tempo

In [172]:
print(pt2list.shape)

(30, 25, 2, 18)


In [173]:
X = pt2list
y = np.array([[str(i)] * int(X.shape[0] / 6) for i in range(6)])
y = y.reshape(y.shape[0] * y.shape[1])
y.shape

(30,)

In [147]:
vaiSVC(pt2list,y)

acuracia: 27.160493827160494 kernel: rbf gamma: 0.001 C: 1
acuracia: 51.23456790123457 kernel: rbf gamma: 0.001 C: 10
acuracia: 61.26543209876543 kernel: rbf gamma: 0.001 C: 100
acuracia: 64.81481481481481 kernel: rbf gamma: 0.001 C: 1000
acuracia: 50.92592592592593 kernel: rbf gamma: 0.01 C: 1
acuracia: 60.802469135802475 kernel: rbf gamma: 0.01 C: 10
acuracia: 64.81481481481481 kernel: rbf gamma: 0.01 C: 100
acuracia: 66.35802469135803 kernel: rbf gamma: 0.01 C: 1000
acuracia: 60.95679012345679 kernel: rbf gamma: 0.1 C: 1
acuracia: 64.81481481481481 kernel: rbf gamma: 0.1 C: 10
acuracia: 66.51234567901234 kernel: rbf gamma: 0.1 C: 100
acuracia: 70.83333333333334 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 62.03703703703704


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(6, 6 - 1) = 5 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


## Parte 2.2 - Domínio da frequência

In [148]:
print(mmdf.shape)
print(fmd.shape)

mmdf = mmdf.swapaxes(0,-1)
fmd = fmd.swapaxes(0,-1)

print(mmdf.shape)
print(fmd.shape)

mmdf = mmdf.reshape(41*30*2,6)
fmd = fmd.reshape(41*30*2,6)

print(mmdf.shape)
print(fmd.shape)

(6, 30, 2, 41)
(6, 30, 2, 41)
(41, 30, 2, 6)
(41, 30, 2, 6)
(2460, 6)
(2460, 6)


# Parte 2.3 - Domínio da frequência e tempo

In [149]:
combfre = np.concatenate((mmdf,fmd,pt2list),axis = 0)
print(combfre.shape)

(7080, 6)


### Criando vetor de labels

In [150]:
X = combfre
y = np.array([[str(i)] * int(X.shape[0] / 6) for i in range(6)])
y = y.reshape(y.shape[0] * y.shape[1])
y.shape

(7080,)

## Treinando e testando

In [151]:
vaiSVC(combfre,y)

acuracia: 59.65160075329566 kernel: rbf gamma: 0.001 C: 1
acuracia: 63.653483992467045 kernel: rbf gamma: 0.001 C: 10
acuracia: 64.87758945386064 kernel: rbf gamma: 0.001 C: 100
acuracia: 66.90207156308851 kernel: rbf gamma: 0.001 C: 1000
acuracia: 64.21845574387947 kernel: rbf gamma: 0.01 C: 1
acuracia: 64.07721280602637 kernel: rbf gamma: 0.01 C: 10
acuracia: 65.01883239171374 kernel: rbf gamma: 0.01 C: 100
acuracia: 64.21845574387947 kernel: rbf gamma: 0.01 C: 1000
acuracia: 62.241054613935965 kernel: rbf gamma: 0.1 C: 1
acuracia: 59.981167608286256 kernel: rbf gamma: 0.1 C: 10
acuracia: 56.355932203389834 kernel: rbf gamma: 0.1 C: 100
acuracia: 55.037664783427495 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 44.962335216572505


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(6, 6 - 1) = 5 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


#### FFT

In [46]:


print(datax.shape)
psdf = []
fmn = []
fmd = []
cont = 0
allft = []
for lvl1 in datax:
    ffttemp = []
    fmd_temp = []
    fmn_temp = []
    som_fmd = 0
    som_fmn = 0
    for lvl2 in lvl1:
        for lvl3 in lvl2:
            #, hop_length=150
            if (cont < 12):
                temp = np.abs(stft(lvl3[cont], n_fft=12,hop_length=350))
                psd = welch(temp)
                psdf.append(psd)
                Fi = (cont * 500) / (2 * len(psd[1]))
                cont+=1
            for lvl3 in psd[1]:
                som_fmd += lvl3.sum()
                som_fmn += lvl3.sum()
                
            fmn_temp.append((Fi * som_fmn) / som_fmn)
            fmd_temp.append(0.5 * som_fmd)
            allfttemp = fmn_temp + fmd_temp
    fmd_temp = np.array(fmd_temp)
    fmd.append(fmd_temp)
    fmn_temp = np.array(fmn_temp)
    fmn.append(fmn_temp)
    allft.append(allfttemp)
    
fmd = np.array(fmd)
fmn = np.array(fmn)
psdf = np.array(psdf)
allft = np.array(allft)
print(psdf.shape)
print(fmd.shape)
print(fmn.shape)
#print(allft.shape)

(30, 6, 2, 12, 500)
(12, 2)
(30, 12)
(30, 12)


#### PSD

In [357]:

print(psdf.shape)

(11, 2)


#### FMD

In [358]:

print(fmd.shape)

(6, 60)


#### FMN

In [359]:

print(fmn.shape)

(6, 60)


#### Junção do FMN e FMD, MAV, RMS, VAR

In [360]:
#allft = allft * VARLIST [:,0:120]

print(allft.shape)

(6, 120)


### Movimento 1

In [425]:
teste = np.concatenate((iemg, rmslist, varlist, fmn, fmd), axis =1)
print(teste.shape)
X = teste.reshape(12600,1)
print(X.shape)

(6, 2100)
(12600, 1)


In [454]:
y = []
cont = 0


cumy = []


for i in range(6):
    y0 = []
    cont = 0
    for movimentos in teste:
        
        for sample in movimentos:
            if(cont == i):
                y0.append(1)

            
            else:
                y0.append(2)
        print(cont)
        cont+=1
        print(len(y0))
    cumy.append(y0)
print(cumy[cont-1][10500:])
    

cumy = np.array(cumy)
print(cumy.shape)


0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
0
2100
1
4200
2
6300
3
8400
4
10500
5
12600
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [455]:
#y = []
#for lvl1 in teste:
   #for lvl2 in lvl1:
    #        y.append(lvl2)
#y = np.array(y)
#print(y.shape)

In [456]:
#X = X.swapaxes(0,1)

print(X.shape)
#print(y.shape)
print(cumy[1][655:665])


(12600, 1)
[2 2 2 2 2 2 2 2 2 2]


In [458]:

ss = StandardScaler()
ss.fit(X)
ss.transform(X)
for ypsilons in cumy:
    print(ypsilons.shape)
    X_train, X_test, y_train, y_test = train_test_split(X, ypsilons, test_size = 0.2, shuffle=True)
    #X_train, X_test, y_train, y_test = train_test_split(X, y0, test_size=0.33, random_state=42)
    #clf = OutputCodeClassifier(LinearSVC(random_state=0), code_size=2, random_state=0)
    #clf.fit(X_train, y_train).predict(X_test) 
    #y_pred = clf.fit(X_train, y_train).predict(X_test)
    #print(y_pred.shape)
    #print("\tPrecision: %1.3f" % precision_score(y_test, y_pred))


    for kernel in ['rbf']:#, 'linear']:
        for gamma in [0.001, 0.01, 0.1]:
            for C in [1, 10, 100, 1000]:
                classificador = []
                classificador = svm.SVC(gamma=gamma, C=C, kernel=kernel).fit(X_train, y_train)
                print('acuracia:', (classificador.score(X_test, y_test)) * 100, 'kernel:', kernel, 'gamma:', gamma, 'C:', C)
            

    cls = []
    cls = LinearDiscriminantAnalysis(solver='lsqr', shrinkage='auto', n_components=7).fit(X_train, y_train)
    print('\nLDA acuracia:', cls.score(X_test, y_test) * 100)

(12600,)
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 1
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 10
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 100
acuracia: 83.25396825396825 kernel: rbf gamma: 0.001 C: 1000
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 1
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 10
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 100
acuracia: 83.25396825396825 kernel: rbf gamma: 0.01 C: 1000
acuracia: 83.25396825396825 kernel: rbf gamma: 0.1 C: 1
acuracia: 83.25396825396825 kernel: rbf gamma: 0.1 C: 10
acuracia: 83.25396825396825 kernel: rbf gamma: 0.1 C: 100
acuracia: 83.2936507936508 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 83.25396825396825
(12600,)


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 84.92063492063492 kernel: rbf gamma: 0.001 C: 1
acuracia: 84.92063492063492 kernel: rbf gamma: 0.001 C: 10
acuracia: 87.97619047619048 kernel: rbf gamma: 0.001 C: 100
acuracia: 88.17460317460318 kernel: rbf gamma: 0.001 C: 1000
acuracia: 87.65873015873015 kernel: rbf gamma: 0.01 C: 1
acuracia: 87.10317460317461 kernel: rbf gamma: 0.01 C: 10
acuracia: 88.29365079365078 kernel: rbf gamma: 0.01 C: 100
acuracia: 87.97619047619048 kernel: rbf gamma: 0.01 C: 1000
acuracia: 88.65079365079364 kernel: rbf gamma: 0.1 C: 1
acuracia: 88.76984126984127 kernel: rbf gamma: 0.1 C: 10
acuracia: 88.84920634920634 kernel: rbf gamma: 0.1 C: 100
acuracia: 88.84920634920634 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 84.92063492063492
(12600,)


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 1
acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 10
acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 100
acuracia: 83.01587301587303 kernel: rbf gamma: 0.001 C: 1000
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 1
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 10
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 100
acuracia: 83.01587301587303 kernel: rbf gamma: 0.01 C: 1000
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 1
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 10
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 100
acuracia: 83.01587301587303 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 83.01587301587303
(12600,)


...

/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 1
acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 10
acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 100
acuracia: 82.57936507936508 kernel: rbf gamma: 0.001 C: 1000
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 1
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 10
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 100
acuracia: 82.57936507936508 kernel: rbf gamma: 0.01 C: 1000
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 1
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 10
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 100
acuracia: 82.57936507936508 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 82.57936507936508
(12600,)


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 1
acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 10
acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 100
acuracia: 83.80952380952381 kernel: rbf gamma: 0.001 C: 1000
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 1
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 10
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 100
acuracia: 83.80952380952381 kernel: rbf gamma: 0.01 C: 1000
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 1
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 10
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 100
acuracia: 83.80952380952381 kernel: rbf gamma: 0.1 C: 1000

LDA acuracia: 83.80952380952381


/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:466: ChangedBehaviorWarning: n_components cannot be larger than min(n_features, n_classes - 1). Using min(n_features, n_classes - 1) = min(1, 2 - 1) = 1 components.
  ChangedBehaviorWarning)
/home/higor/.local/lib/python3.7/site-packages/sklearn/discriminant_analysis.py:472: FutureWarning: In version 0.23, setting n_components > min(n_features, n_classes - 1) will raise a ValueError. You should set n_components to None (default), or a value smaller or equal to min(n_features, n_classes - 1).
  warnings.warn(future_msg, FutureWarning)


In [ ]:


clf = SVC(gamma='auto')

X = teste
y = ["movimento1", "movimento2", "movimento3", "movimento4", "movimento5", "movimento6"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
clf.fit(X, y) 


In [395]:
print(clf.predict(X_test))
print(X_test)

['movimento1' 'movimento2']
[[1.68089912e-01 1.68753008e-01 1.71609190e-01 ... 7.96965344e+04
  8.09700791e+04 8.22436237e+04]
 [1.74516332e-01 1.41415272e-01 1.74720424e-01 ... 7.38655907e+04
  7.51391354e+04 7.64126801e+04]]
